In [1]:
import geopandas as gpd
import numpy as np
import networkx as nx

#### Module 0 work

In [ ]:
class Map():
    
    def __init__(self,geopackage_path):
        self.roads = self._load_roads(geopackage_path)
        self.graph = self._load_graph(geopackage_path)
        
    def _load_roads(self,geopackage_path):
        # load map info
        roads_gdf = gpd.read_file(geopackage_path, layer='edges')
        # find traffic flow as vehicles per minute per lane
        roads_gdf['traffic_flow'] = roads_gdf['aadt']/roads_gdf['lanes']/12/60 
        # calculate current speed
        roads_gdf['current_speed'] = roads_gdf['max_speed'] - (roads_gdf['max_speed'] / 220 ) * roads_gdf['traffic_flow']
        roads_gdf['current_speed'] = max(roads_gdf['current_speed'], 0)
        return roads_gdf
    
    def _load_graph(self, geopackage_path):
        
        # Load Info
        roads_gdf = gpd.read_file(geopackage_path, layer='edges')
        nodes_gdf = gpd.read_file(geopackage_path, layer='nodes')
        
        # Create a graph from the road network data
        G = nx.Graph()
        for i,row in roads_gdf.iterrows():
            G.add_edge(row['node1_id'], row['node2_id'], weight=row['length'], roadid=row['road_id'])
            pos = {
            row['node1_id']: nodes_gdf.loc[row['node1_id'],'geometry'],
            row['node2_id']: nodes_gdf.loc[row['node2_id'],'geometry']
            }
            nx.set_node_attributes(G, pos, 'geometry')
    
    
    def Experience_Snow(self, inches_of_snow):
        '''
        Reduce the road max speed by 12% per inch of snow if not plowed.
        Set all roads plow time to inf
        '''
        snow_effect = (1 - 0.12) ** inches_of_snow
        self.roads['speed_after_snow'] = self.roads['current_speed']*= snow_effect
        self.roads['plow_time'] = np.inf
        
 

In [ ]:
roads_gdf

In [ ]:
nodes_gdf = gpd.read_file(geopackage_path, layer='nodes')
nodes_gdf.head(3)

In [ ]:
nodes_gdf.loc[0,'geometry'].coords[0]

In [ ]:
roads_gdf = gpd.read_file(geopackage_path, layer='edges')
roads_gdf.head(3)

In [ ]:
road_network = gpd.read_file("network_shapefile/edges.shp")

In [ ]:
road_network.head(3)

In [ ]:
# Create a graph from the road network data
    G = nx.Graph()
    for row in road_network.itertuples():
        G.add_edge(row[11], row.to, weight=row.length, osmid=row.osmid)
        pos = {
            row[11]: (row.geometry.bounds[0], row.geometry.bounds[1]),
            row.to: (row.geometry.bounds[2], row.geometry.bounds[3])
        }
        nx.set_node_attributes(G, pos, 'geometry')

In [ ]:
G = nx.Graph()
for row in road_network.itertuples():
    G.add_edge(row[11], row.to, weight=row.length, osmid=row.osmid)

In [ ]:
print(G)

In [ ]:
row

In [ ]:
row[11]

#### Module 0 Run

In [6]:
class Map():
    
    def __init__(self,geopackage_path):
        self.roads = self._load_roads(geopackage_path)
        self.graph = self._load_graph(geopackage_path)
        
    def _load_roads(self,geopackage_path):
        # load map info
        roads_gdf = gpd.read_file(geopackage_path, layer='edges')
        # find traffic flow as vehicles per minute per lane
        roads_gdf['traffic_flow'] = roads_gdf['aadt']/roads_gdf['lanes']/12/60 
        # calculate current speed
        roads_gdf['normal_speed'] = roads_gdf['max_speed'] - (roads_gdf['max_speed'] / 220 ) * roads_gdf['traffic_flow']
        return roads_gdf
    
    def _load_graph(self, geopackage_path):
        
        # Load Info
        roads_gdf = gpd.read_file(geopackage_path, layer='edges')
        nodes_gdf = gpd.read_file(geopackage_path, layer='nodes')
        
        # Create a graph from the road network data
        G = nx.Graph()
        for i,row in roads_gdf.iterrows():
            G.add_edge(row['node1_id'], row['node2_id'], weight=row['length'], roadid=row['road_id'])
            pos = {
            row['node1_id']: nodes_gdf.loc[row['node1_id'],'geometry'],
            row['node2_id']: nodes_gdf.loc[row['node2_id'],'geometry']
            }
            nx.set_node_attributes(G, pos, 'geometry')
        return G
    
    def Experience_Snow(self, inches_of_snow=4):
        '''
        Reduce the road max speed by 12% per inch of snow if not plowed.
        Set all roads plow time to inf
        '''
        snow_effect = (1 - 0.12) ** inches_of_snow
        self.roads['speed_after_snow'] = self.roads['normal_speed']* snow_effect
        self.roads['plow_time'] = np.inf
        
 

In [7]:
M = Map('AA_Map_Final.gpkg')
M.roads.head(5)

,road_id,length,max_speed,aadt,lanes,node1_id,node2_id,geometry,traffic_flow,normal_speed
0,4803,0.19,55,12605,3,119,133,"LINESTRING (-83.74491 42.30368, -83.74487 42.3...",5.835648,53.541088
1,9072,0.03,35,3346,2,309,133,"LINESTRING (-83.74336 42.30504, -83.74336 42.3...",2.323611,34.630335
2,9066,0.08,25,7692,2,417,119,"LINESTRING (-83.74628 42.30430, -83.74613 42.3...",5.341667,24.392992
3,9125,0.15,55,12605,3,394,119,"LINESTRING (-83.74639 42.30148, -83.74632 42.3...",5.835648,53.541088
4,9272,0.14,40,4069,1,413,119,"LINESTRING (-83.74642 42.30466, -83.74517 42.3...",5.651389,38.972475


In [9]:
M.graph.nodes

NodeView((119, 133, 309, 417, 394, 413, 252, 228, 65, 10, 334, 40, 218, 420, 253, 178, 259, 365, 189, 270, 150, 82, 132, 265, 393, 273, 128, 210, 414, 313, 103, 236, 89, 173, 197, 280, 403, 318, 75, 304, 234, 64, 289, 347, 53, 117, 209, 149, 1, 71, 157, 115, 406, 303, 51, 187, 207, 125, 107, 7, 3, 208, 57, 250, 242, 190, 397, 204, 267, 145, 134, 161, 277, 15, 249, 411, 18, 324, 62, 67, 47, 153, 415, 144, 201, 206, 327, 94, 335, 73, 22, 257, 317, 373, 93, 126, 263, 37, 76, 358, 91, 63, 354, 333, 419, 172, 308, 292, 345, 121, 290, 375, 102, 160, 239, 42, 118, 287, 361, 77, 141, 32, 155, 383, 262, 110, 388, 196, 179, 370, 193, 366, 367, 46, 258, 213, 297, 142, 108, 384, 401, 12, 288, 302, 194, 13, 237, 412, 295, 369, 321, 159, 17, 14, 162, 123, 284, 400, 200, 181, 44, 38, 29, 389, 248, 404, 151, 286, 283, 0, 332, 350, 346, 135, 56, 158, 169, 202, 353, 282, 359, 238, 171, 307, 409, 314, 165, 368, 95, 223, 114, 83, 407, 176, 360, 216, 322, 98, 266, 382, 342, 148, 52, 85, 405, 348, 116, 340,

In [ ]:
M.Experience_Snow(4)
M.roads.head(5)

In [ ]:
M.Experience_Snow(6)
M.roads.head(5)